In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

# prevent tensorflow from using GPU. Otherwise, run out of memory
# https://stackoverflow.com/questions/44552585/prevent-tensorflow-from-accessing-the-gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2")

In [2]:
# FUNCTIONS

# Create tokens with simple split on whitespace
def simple_token(s):
    return s.split()

# Get token length
def token_length(s):
    return len(s)

# Create "embedding lists" of equal size -- pad with empty characters, e.g. ""
# https://stackoverflow.com/questions/24066904/most-pythonic-way-to-extend-a-list-to-exactly-a-certain-length
def pad_list(some_list, target_len):
    return some_list[:target_len] + [""]*(target_len - len(some_list))

# Get the elmo embeddings
def elmo_tweet_embedder(tokens,len_list):
    
    tokens_input = tokens #load a tweet
    tokens_length = len_list # get length of tweet

    #create embedding
    embedding_tensor = elmo(inputs={"tokens":tokens_input,"sequence_len":tokens_length},
                            signature="tokens", as_dict=True)["word_emb"] # <-- passing in a list instead of [word]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        embedding = sess.run(embedding_tensor)
        return embedding

# combine all together
def make_embeddings(df, max_seq_len):
    m = max_seq_len
    
    # create a token column in df
    df['tokens'] = df['text'].apply(simple_token)
    
    # pad the tokens
    df['tokens'] = df.apply(lambda x: pad_list(x['tokens'],m),axis=1)
    
    # split data into smaller batches of size 100. http://bit.ly/2P4J8HJ
    text_batches = [df['tokens'][i:i+100] for i in range(0,df.shape[0],100)]
    len_lists = [[m] * len(x) for x in text_batches]

    # create list of sentiments (y values)
    y = df['sent'].to_numpy()
    
    # make embeddings
    embeddings = []
    for i in range(0,len(text_batches)):
        elmo_train = elmo_tweet_embedder(text_batches[i].tolist(),len_lists[i])
        embeddings.append(elmo_train)
        
    embeddings = np.concatenate(embeddings, axis=0)
    
    return embeddings, y

In [9]:
path = '/home/tim/Documents/Sentiment/Data/processed/'
file_list = os.listdir(path)

# create the data set names
names = []
for f in file_list:
    f = f.split('.')
    l = ["x_"+f[0],"y_"+f[0]]
    names.append(l)

data_names = names
    
# index through files and create x and y sets
for i in range(0,len(file_list)):
    
    s = names
   
    p = os.path.join(path,file_list[i])
    col_names = ['text','sent']
    df = pd.read_csv(p,delimiter="\t",names=col_names)

    print(s[i][0], "start")
    s[i][0], s[i][1] = make_embeddings(df, 80) 

x_train = s[0][0]
y_train = s[0][1]
x_dev = s[1][0]
y_dev = s[1][1]
x_test = s[2][0]
y_test = s[2][1]

x_train start
x_dev start
x_test start


In [15]:
# Create Pickles
pickle_path = '/home/tim/Documents/Sentiment/Data/pickles'

# x_train
pickle_out = open(os.path.join(pickle_path,'x_train.pickle'),"wb")
pickle.dump(x_train, pickle_out)
pickle_out.close()

# y_train
pickle_out = open(os.path.join(pickle_path,'y_train.pickle'),"wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

# x_dev
pickle_out = open(os.path.join(pickle_path,'x_dev.pickle'),"wb")
pickle.dump(x_dev, pickle_out)
pickle_out.close()

# y_dev
pickle_out = open(os.path.join(pickle_path,'y_dev.pickle'),"wb")
pickle.dump(y_dev, pickle_out)
pickle_out.close()

# x_test
pickle_out = open(os.path.join(pickle_path,'x_test.pickle'),"wb")
pickle.dump(x_test, pickle_out)
pickle_out.close()

# y_test
pickle_out = open(os.path.join(pickle_path,'y_test.pickle'),"wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()